In [1]:
# 2 : 키보드 's' 입력 시 초기 설정( 완성본 )

import cv2 as cv
import numpy as np
import mediapipe as mp
import math
import time

# 렌더링 용 얼굴 좌표
FOR_3D = [33 ,263, 6, 61, 291, 199]
# 얼굴 회전 각도 추출에 사용할 모든 얼굴 좌표 정보
FACE=list(range(468))
# 왼 쪽, 오른 쪽 눈 상하좌우 좌표 정보
LEFT_EYE_TOP_BOTTOM_LEFT_RIGHT = [386, 374,263, 362]
RIGHT_EYE_TOP_BOTTOM_LEFT_RIGHT = [159, 145,133, 33]

# 렌더링 함수
def draw_landmark(height, width, outputs, land_mark, color):
    for face in land_mark:
        point = outputs.multi_face_landmarks[0].landmark[face]
        # 화면 비율에 맞게 좌표 값 위치 정보 수정
        point_scale = ((int)(point.x * width), (int)(point.y * height))
        cv.circle(image, point_scale, 2, color, 1)
        
# 두 점 사이의 거리 측정 함수
def cal_distance(height, width, camera_point1, camera_point2):
    point1 = int(camera_point1.x * width), int(camera_point1.y * height)
    point2 = int(camera_point2.x * width), int(camera_point2.y * height)
    distance = math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)
    return distance

# 가로 세로 비율 계산
def cal_ratio(height,width, outputs, left, right, secant):
    landmark = outputs.multi_face_landmarks[0]
    top1 = landmark.landmark[left[0]]
    bottom1 = landmark.landmark[left[1]]
    top_bottom_dis1 = cal_distance(height,width, top1, bottom1)

    top2 = landmark.landmark[right[0]]
    bottom2 = landmark.landmark[right[1]]
    top_bottom_dis2 = cal_distance(height,width, top2, bottom2)

    lwide1 = landmark.landmark[left[2]]
    rwide1 = landmark.landmark[left[3]]
    lwide_rwide_dis1 = cal_distance(height,width,lwide1, rwide1)

    lwide2 = landmark.landmark[right[2]]
    rwide2 = landmark.landmark[right[3]]
    lwide_rwide_dis2 = cal_distance(height,width,lwide2, rwide2)

    sum_wide = 0
    sum_height = 0

    # 가로 길이 보정
    # 얼굴 회전에 따라 왼 쪽 눈 가로 길이와 오른 쪽 가로 길이의 평균 값을 사용하려 하였으나,
    # 얼굴을 회전 시 코가 한 쪽 눈의 가로 길이 측정을 막는 문제가 있어, 두 길이의 비율 차이가 0.1 이상 커지면 한 쪽 눈의 가로 길이로만 측정하는 것으로 바꿨다.
    if lwide_rwide_dis1!=0 and lwide_rwide_dis2 != 0: 
        if lwide_rwide_dis1/lwide_rwide_dis2 > 1.1:
            sum_wide = round(lwide_rwide_dis1/2)*2
        elif lwide_rwide_dis1 /lwide_rwide_dis2 < 0.9:
             sum_wide = round(lwide_rwide_dis2/2)*2
        else:
            sum_wide = (round(lwide_rwide_dis1/3)*3+round(lwide_rwide_dis2/3)*3)/2
    
    # 세로 길이 보정
    if top_bottom_dis1 !=0 and top_bottom_dis2 != 0: 
        if top_bottom_dis1/top_bottom_dis2 > 1.1:
            sum_height = round(top_bottom_dis1/2)*2
        elif top_bottom_dis1/top_bottom_dis2 < 0.9:
            sum_height = round(top_bottom_dis2/2)*2
        else:
            sum_height =(round(top_bottom_dis1/2)*2+round(top_bottom_dis2/2)*2)/2
    
    # 회전 각 secant를 사용해 비율 측정
    secant_u = math.ceil(secant*100)/100
    secant_d = math.floor(secant*100)/100
    # 눈 좌표 값 추출시 미세한 오차들이 많이 발생하는 것을 발견하고 그 미세한 오차들을 줄이고자 반올림 함수를 활용해 식을 만듬
    aspect_ratio = round((round((sum_height*secant_u*secant_u*secant_u)/(sum_wide*secant_d*secant_d)/2, 3)
                        * 2 + round((sum_height*secant_u*secant_u*secant_u)/(sum_wide*secant_d*secant_d)/5, 2)*5) /2, 4)

    cv.putText(image, "ratio: " + str(aspect_ratio), (100, 90), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    return aspect_ratio

def get_3D(height, width, outputs, landmark):
    face_3d = []
    face_2d = []

    for face in landmark:
        point = outputs.multi_face_landmarks[0].landmark[face]
        x, y = int(point.x * width), int(point.y * height)
        # 2d 좌표 값과 3d 좌표 값 추출
        face_2d.append([x, y])
        face_3d.append([x, y, point.z])       

    face_2d = np.array(face_2d, dtype=np.float64)
    face_3d = np.array(face_3d, dtype=np.float64)
    # 카메라 행렬 설정
    # focal length를 구했으나 예상하던 값과 너무 달라 사용하지 못해음
    # 임의로 설정할 때는 화면의 width 값과 height 값을 쓴다고 하여 이것을 사용
    focal_length = width 
    cam_matrix = np.array([ [focal_length, 0, height / 2],
                            [0, focal_length, width / 2],
                            [0, 0, 1]])
    # 왜곡 행렬 또한 focal length를 구할 때 알 수 있으나, 사용하지 않으므로 0으로 설정
    dist_matrix = np.zeros((4, 1), dtype=np.float64)
    success, rot_vec, trans_vec = cv.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)
    # 회전 행렬 구함
    rmat, jac = cv.Rodrigues(rot_vec)
    # 회전 각도 구함
    angles, mtxR, mtxQ, Qx, Qy, Qz = cv.RQDecomp3x3(rmat)
    # 회전 각도 단위 변경
    x = angles[0] * 360 * math.pi

    str_x = str(np.round(x,2))
    cv.putText(image, "x: " + str_x, (500, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    return x

##    
face_mesh = mp.solutions.face_mesh
face_model = face_mesh.FaceMesh(max_num_faces= 1,
                                refine_landmarks=True,
                                min_detection_confidence=0.5,
                                min_tracking_confidence=0.5)
##
cap = cv.VideoCapture(0)

##
face_updown_angle = []

# 초기 안 졸 때의 눈의 비율
set_ratio = 0
# 선형 회귀로 구한 졸음 판단 기준 비율 값
ratio_threshold = 1.07 * 0.3334485666104562 - 0.08
# 위 ratio_threshold를 사용해 구한 졸음 판단 기준 시간 값
time_threshold = 1.2170644936682
# ratio_threshold 이하로 떨어 졌을 경우 시간 측정을 위한 변수
time_check = []
# threshold가 아닌 209 frame 당 졸음 상태, 안 졸음 상태 비율 측정
predict_result = []
real_predict_result = []
while cap.isOpened():
    result, image = cap.read()

    if not result:
        print("Ignoring empty camera frame.")
        break
    else:

        set_ratio = 1


        height, width =image.shape[:2]
        image.flags.writeable = False
        image = cv.cvtColor(cv.flip(image, 1), cv.COLOR_BGR2RGB)
        outputs = face_model.process(image)

        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

        if outputs.multi_face_landmarks:
            ##
            draw_landmark(height, width,outputs, FOR_3D, (0,255,0))
            draw_landmark(height, width,outputs, LEFT_EYE_TOP_BOTTOM_LEFT_RIGHT, (0,0,255))
            draw_landmark(height, width,outputs, RIGHT_EYE_TOP_BOTTOM_LEFT_RIGHT, (0,0,255))

            get_face_angle = get_3D(height, width,outputs,FACE)
            th = math.radians(get_face_angle)
            secant = 1.0/math.cos(th)
            ratio =  cal_ratio(height, width,outputs, LEFT_EYE_TOP_BOTTOM_LEFT_RIGHT, RIGHT_EYE_TOP_BOTTOM_LEFT_RIGHT,secant)
           
            if (len(predict_result) == 209):
                real_predict_result.append((sum(predict_result)/len(predict_result)))
                predict_result.pop(0)
            
            if(set_ratio!=0):
                # ratio_intv = set_ratio - ratio # 초기 설정된 눈 비율과 현재 측정된 눈 비율의 차이
                if (ratio <= ratio_threshold): # 초기 눈 비율과의 차이가 0.055(임의) 보다 크면 눈은 얇게 뜨거나 감은 경우
                    predict_result.append(1)
                    Time = time.time() # 시간 확인
                    time_check.append(round(Time,3)) # 시간 저장(얼마나 얇게 뜨는지 측정하기 위함)

                elif (ratio > ratio_threshold): # 초기 눈 비율과의 차이가 0.055(임의) 보다 작으면 = 눈을 뜬 경우
                    predict_result.append(0)
                    time_check.clear() # 저장된 시간 지우기(깨어났다고 판단)

                if not time_check: # 저장된 시간이 없을 경우(눈을 제대로 떳을 경우)
                    pass
                else:
                    time_difference = time_check[-1] - time_check[0] # 눈을 작게 뜬 시간 측정
                    if (time_difference >=time_threshold and time_difference <2.5): # 시간이 time_threshold 이상 2.5초 미만(임의)일 때
                        face_updown_angle.append(round(get_face_angle,3)) # 얼굴 각도 저장
                        angle_difference = face_updown_angle[-1] - face_updown_angle[0] # 얼굴 각도 변화 측정
                        cv.putText(image,"drowsy", (100,50), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255,0,0), 3)
                        if ( angle_difference > 0.8): # 얼굴 각도 변화가 0.8도(임의) 이상일 때 
                            cv.putText(image,"dangerous", (100,50), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255,0,0), 3) # 졸림으로 판단
                    elif (time_difference>=2.5): # 눈을 작게 뜬 시간이 2.5초 이상일 때
                        cv.putText(image, "dangerous", (100,50), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255,0,0), 3) # 잔다고 판단
            
            
        cv.imshow("FACE MESH", image)
        key = cv.waitKey(2)
        if key==ord('s') or key ==ord('S'): # s를 누르면 초기 눈 비율 설정
            print("okay")
            print("현재 눈 비율(각도를 고려한)",ratio)
            set_ratio = ratio # 초기 눈 비율 설정
            ratio_threshold = 1.077*set_ratio - 0.071
            print(ratio_threshold) 
        elif key==ord('q') or key ==ord('Q'): # q를 누르면 종료
            break
cap.release()
cv.destroyAllWindows()
print(sum(real_predict_result)/len(real_predict_result))
print(len(real_predict_result))


0.11986410072534434
814


In [4]:
# 영상 normal 눈 평균값으로 초기값 설정 
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split

DATA_PATH_2 = os.path.join('D:\\abc','MP_Data_2_different_noblink_6') 
actions = np.array(['normal','drowsy'])
no_sequences = 30
label_map = {label:num for num, label in enumerate(actions)}
sequences, labels = [], [] 

arr_2 =  [[0 for j in range(no_sequences)] for i in range(len(actions))]
for idx, lm in enumerate(actions):
    for sequence in range(no_sequences) :  # 5번 반복(영상 수)
        res_2 = np.load(os.path.join(DATA_PATH_2, lm, "{}.npy".format(str(sequence+1))))
        arr_2[idx][sequence] = res_2
        
arr_2 = np.array(arr_2)
arr_2 = arr_2[0]

In [13]:
# 3 : threshold 이하 상태 시간 평균값 구하기

import cv2 as cv
import numpy as np
import mediapipe as mp
import math
import time

FOR_3D = [33 ,263, 6, 61, 291, 199]
FACE=list(range(468))

LEFT_EYE_TOP_BOTTOM_LEFT_RIGHT = [386, 374,263, 362]
RIGHT_EYE_TOP_BOTTOM_LEFT_RIGHT = [159, 145,133, 33]

def draw_landmark(height, width, outputs, land_mark, color):
    for face in land_mark:
        point = outputs.multi_face_landmarks[0].landmark[face]
        point_scale = ((int)(point.x * width), (int)(point.y * height))
        cv.circle(image, point_scale, 2, color, 1)
        
def cal_distance(height, width, camera_point1, camera_point2):
    point1 = int(camera_point1.x * width), int(camera_point1.y * height)
    point2 = int(camera_point2.x * width), int(camera_point2.y * height)
    distance = math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)
    return distance

def cal_ratio(height,width, outputs, left, right, secant):
    landmark = outputs.multi_face_landmarks[0]
    top1 = landmark.landmark[left[0]]
    bottom1 = landmark.landmark[left[1]]
    top_bottom_dis1 = cal_distance(height,width, top1, bottom1)

    top2 = landmark.landmark[right[0]]
    bottom2 = landmark.landmark[right[1]]
    top_bottom_dis2 = cal_distance(height,width, top2, bottom2)

    lwide1 = landmark.landmark[left[2]]
    rwide1 = landmark.landmark[left[3]]
    lwide_rwide_dis1 = cal_distance(height,width,lwide1, rwide1)

    lwide2 = landmark.landmark[right[2]]
    rwide2 = landmark.landmark[right[3]]
    lwide_rwide_dis2 = cal_distance(height,width,lwide2, rwide2)

    sum_wide = 0
    sum_height = 0

    if lwide_rwide_dis1!=0 and lwide_rwide_dis2 != 0: # 가로 길이 보정
        if lwide_rwide_dis1/lwide_rwide_dis2 > 1.1 or lwide_rwide_dis1 /lwide_rwide_dis2 < 0.9:
            if lwide_rwide_dis1 > lwide_rwide_dis2:
                sum_wide = round(lwide_rwide_dis1/2)*2
            else:
                sum_wide = round(lwide_rwide_dis2/2)*2
        else:
            sum_wide = (round(lwide_rwide_dis1/3)*3+round(lwide_rwide_dis2/3)*3)/2
    
    if top_bottom_dis1 !=0 and top_bottom_dis2 != 0: # 세로 길이 보정
        if top_bottom_dis1/top_bottom_dis2 > 1.1 or top_bottom_dis1/top_bottom_dis2 < 0.9:
            if top_bottom_dis1 > top_bottom_dis2:
                sum_height = round(top_bottom_dis1/2)*2
            else:
                sum_height = round(top_bottom_dis2/2)*2
        else:
            sum_height =(round(top_bottom_dis1/2)*2+round(top_bottom_dis2/2)*2)/2
    
    secant_u = math.ceil(secant*100)/100
    secant_d = math.floor(secant*100)/100
    aspect_ratio = round((round((sum_height*secant_u*secant_u*secant_u)/(sum_wide*secant_d*secant_d)/2, 3)
                        * 2 + round((sum_height*secant_u*secant_u*secant_u)/(sum_wide*secant_d*secant_d)/5, 2)*5) /2, 4)

    # aspect_ratio = round(sum_height/sum_wide/2*secant,2)*2
    cv.putText(image, "ratio: " + str(aspect_ratio), (100, 90), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    return aspect_ratio

def get_3D(height, width, outputs, landmark):
    face_3d = []
    face_2d = []

    for face in landmark:
        point = outputs.multi_face_landmarks[0].landmark[face]
        x, y = int(point.x * width), int(point.y * height)
        face_2d.append([x, y])
        face_3d.append([x, y, point.z])       

    face_2d = np.array(face_2d, dtype=np.float64)
    face_3d = np.array(face_3d, dtype=np.float64)
    focal_length = width
    cam_matrix = np.array([ [focal_length, 0, height / 2],
                            [0, focal_length, width / 2],
                            [0, 0, 1]])
    dist_matrix = np.zeros((4, 1), dtype=np.float64)
    success, rot_vec, trans_vec = cv.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)
    rmat, jac = cv.Rodrigues(rot_vec)
    angles, mtxR, mtxQ, Qx, Qy, Qz = cv.RQDecomp3x3(rmat)
    x = angles[0] * 360 * math.pi  * 0.75+7
    x = np.round(x,1)
    cv.putText(image, "x: " + str(x), (100, 20), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # 40도 부근에서 값의 변화가 크게 나타나 40도 를 넘어갈 경우 40도로 설정(곡선으로 된 안구 특성, 코의 존재로 한 쪽 눈 측정 불가 등의 이유로 추측)
    if x > 40 :
        x = 40
    elif x <-40:
        x = -40 
    return x

##    
face_mesh = mp.solutions.face_mesh
face_model = face_mesh.FaceMesh(max_num_faces= 1,
                                refine_landmarks=True,
                                min_detection_confidence=0.5,
                                min_tracking_confidence=0.5)
time_threshold_list = []

for i in range(31,34):
    a =str(i)+"_1.mp4"
    ## 영상 입력
    name = a
    cap = cv.VideoCapture(name)

    ## 영상 이름 인식
    if(name[1]!='_'):
        im = name[0]+name[1]  
        sequence = int(im) # '1_2.mp4' 에서 '1' -> 비디오 순서
        if int(name[3]) - 1 == 0:
            action = actions[0] # '1_2.mp4' 에서 '2' -> 비디오 분류(normal or drowsy)
        else:
            action = actions[1]
    else:
        sequence = int(name[0]) # '1_2.mp4' 에서 '1' -> 비디오 순서
        if int(name[2]) - 1 == 0:
            action = actions[0] # '1_2.mp4' 에서 '2' -> 비디오 분류(normal or drowsy)
        else:
            action = actions[1]
            
    ##
    time_check = []
    ratio_mean= []
    threshold = 0.87231284*arr_2[i-1] -0.0018257729
    print(threshold)
    time_threshold = []

    while cap.isOpened():
        result, image = cap.read()
        if not result:
            print("Ignoring empty camera frame.")
            break
        else:
            height, width =image.shape[:2]
            image.flags.writeable = False
            image = cv.cvtColor(cv.flip(image, 1), cv.COLOR_BGR2RGB)
            outputs = face_model.process(image)

            image.flags.writeable = True
            image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

            if outputs.multi_face_landmarks:
                ##
                draw_landmark(height, width,outputs, LEFT_EYE_TOP_BOTTOM_LEFT_RIGHT, (0,0,255))
                draw_landmark(height, width,outputs, RIGHT_EYE_TOP_BOTTOM_LEFT_RIGHT, (0,0,255))

                get_face_angle = get_3D(height, width,outputs,FACE)
                th = math.radians(get_face_angle)
                secant = 1.0/math.cos(th)

                ratio =  cal_ratio(height, width,outputs, LEFT_EYE_TOP_BOTTOM_LEFT_RIGHT, RIGHT_EYE_TOP_BOTTOM_LEFT_RIGHT,secant)
            
                if (ratio <= threshold): # 초기 눈 비율과의 차이가 0.055(임의) 보다 크면 눈은 얇게 뜨거나 감은 경우
                    cv.putText(image,"threshold", (100,50), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255,0,0), 3) # 졸림으로 판단
                    Time = time.time() # 시간 확인
                    time_check.append(round(Time,3)) # 시간 저장(얼마나 얇게 뜨는지 측정하기 위함)

                elif (ratio > threshold) & (len(time_check)>= 2): # 초기 눈 비율과의 차이가 0.055(임의) 보다 작으면 = 눈을 뜬 경우
                    time_difference = time_check[-1] - time_check[0]
                    if (time_difference >= 0.3) : # 눈 깜빡이는 평균 시간 약 0.2초 이하 되는 시간차는 제외한다.
                        time_threshold.append(time_difference)
                    time_check.clear() # 저장된 시간 지우기(깨어났다고 판단)
    
            cv.imshow("FACE MESH", image)
            key = cv.waitKey(2)

            if key==ord('q') or key ==ord('Q'):
                break
    cap.release()
    cv.destroyAllWindows()

    if len(time_threshold) != 0:
        mean = sum(time_threshold)/len(time_threshold)
        print(i," : ",mean)
        time_threshold_list.append(mean)
    else:
        print("no data")
time_threshold_mean = sum(time_threshold_list) / len(time_threshold_list)
print("time_threshold_list", time_threshold_list)
print("time_threshold_mean", time_threshold_mean)

0.29655546130277044
Ignoring empty camera frame.
1  :  0.64933332942781
0.316230326535513
Ignoring empty camera frame.
2  :  0.863250023788876
0.3168285091997644
Ignoring empty camera frame.
3  :  2.8817333380381265
0.3335821386485887
Ignoring empty camera frame.
4  :  0.7616922855377197
0.31969600287114447
Ignoring empty camera frame.
5  :  1.1144500017166137
0.2578022284688928
Ignoring empty camera frame.
6  :  0.7425000326974052
0.26479520339144036
Ignoring empty camera frame.
7  :  0.9146666526794434
0.30742271328462645
Ignoring empty camera frame.
8  :  3.9721818187020044
0.3539305288637979
Ignoring empty camera frame.
9  :  0.9543845836932843
0.2991406649334613
Ignoring empty camera frame.
10  :  1.025600004196167
0.3142435377244306
Ignoring empty camera frame.
11  :  1.790928534099034
0.28917407855446753
Ignoring empty camera frame.
12  :  3.033899998664856
0.26208252427939704
Ignoring empty camera frame.
13  :  1.105666610929701
0.3347775811747115
Ignoring empty camera frame.
1